In [3]:
import glob 
import numpy as np
from PIL import Image
import os
import shutil
import random 
import zipfile
from pathlib import Path
import Utils

import tensorflow as tf
import tensorflow.keras.layers as layers
import tensorflow.keras.optimizers as optimizers
import tensorflow.keras.losses as losses
import tensorflow.keras.metrics as metrics
import tensorflow.keras.models as models
import tensorflow.keras.regularizers as regularizers
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.applications import ResNet101V2, ResNet152V2, ResNet50V2
from tensorflow.keras.applications.efficientnet import EfficientNetB5

from tensorflow.python.keras import backend as K

In [12]:
def configure_tf_gpu():
    # Reduce logging output.
    os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'
    tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.INFO)
    config = tf.compat.v1.ConfigProto()
    # dynamically grow the memory used on the GPU
    config.gpu_options.allow_growth = True
    # Allowing TF to automatically choose an existing and
    # supported device to run the operations in case the specified one doesn't exist
    config.allow_soft_placement = True
    # to log device placement (on which device the operation ran)
    config.log_device_placement = False
    config.gpu_options.per_process_gpu_memory_fraction = 0.9
    # (nothing gets printed in Jupyter, only if you run it standalone)
    sess = tf.compat.v1.Session(config=config)
    # set this TensorFlow session as the default session for Keras
    K.set_session(sess)

def build_model(shape):
    base_model = EfficientNetB5(include_top=False, weights='imagenet', input_shape=shape, pooling='max')
    base_model.trainable=False
    model = models.Sequential()
    model.add(base_model)   
    model.add(layers.BatchNormalization(axis = -1 , momentum = 0.99 , epsilon = 0.001))
    model.add(layers.Dense(256, kernel_regularizer = regularizers.l2(l=0.016) , activity_regularizer=regularizers.l1(0.006),
         bias_regularizer= regularizers.l1(0.006) , activation = 'relu'))
    model.add(layers.Dropout(0.4))
    model.add(layers.Dense(1, activation='sigmoid'))
    model.compile(optimizer=optimizers.Adamax(learning_rate=1e-3), loss='binary_crossentropy', metrics=['accuracy'])
    return model

def prepare_images(root_path, seed, test_size=0.3):
    if os.path.isdir(root_path + '/train/cat') or os.path.isdir(root_path + 'test/cat'):
        shape = Image.open(glob.glob(root_path + '/train/cat/*.jpg')[0]).size
        return root_path + '/train', root_path + '/test', shape
    os.makedirs(root_path + '/train/cat')
    os.makedirs(root_path + '/train/dog')
    os.makedirs(root_path + '/test/cat')
    os.makedirs(root_path + '/test/dog')
    files = glob.glob(root_path + '/*.jpg')
    shape = Image.open(files[0]).size
    
    N = len(files)
    n_train = int((1-test_size)*N)
    random.seed(42)
    population = range(N)
    train_indices = random.sample(population, n_train)
    test_indices = list(set(population).difference(set(train_indices)))
    ## train images
    for i in train_indices:
        file = files[i]
        if 'cat' in file:
            shutil.move(file, root_path + '/train/cat/' + str(i) + '.jpg')
        if 'dog' in file:
            shutil.move(file, root_path + '/train/dog/' + str(i) + '.jpg')
    ## test images
    for i in test_indices:
        file = files[i]
        if 'cat' in file:
            shutil.move(file, root_path + '/test/cat/' + str(i) + '.jpg')
        if 'dog' in file:
            shutil.move(file, root_path + '/test/dog/' + str(i) + '.jpg')   
    return root_path + '/train', root_path + '/test', shape

def unzip_images(zip_path):
    if not os.path.isfile(zip_path):
        return -1
    root_dir = str(Path(zip_path).parent)
    if os.path.isdir(root_dir + '/Image_set/'):
        return root_dir + '/Image_set/train/'
    extract_path = root_dir + '/Image_Set/'
    with zipfile.ZipFile(zip_path, 'r') as zip_ref:
        zip_ref.extractall(root_dir + '/Image_Set/')
    if os.path.isfile(extract_path + 'train.zip'):
        with zipfile.ZipFile(extract_path + 'train.zip', 'r') as zip_ref:
            zip_ref.extractall(extract_path)
    return extract_path + '/train/'        
        

In [5]:
### Download Images
zip_path = 'dogs-vs-cats.zip'
if not os.path.isfile(zip_path):
    Utils.download_file_from_google_drive(zip_path)

In [6]:
### Extract zip files and prepare images
root_dir = unzip_images(zip_path)
seed=42
img_size = (224 , 224, 3)
train_path, test_path, shape = prepare_images(root_dir, seed=seed, test_size=0.2)

In [13]:
model = build_model(img_size)
model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 efficientnetb5 (Functional)  (None, 2048)             28513527  
                                                                 
 batch_normalization_2 (Batc  (None, 2048)             8192      
 hNormalization)                                                 
                                                                 
 dense_4 (Dense)             (None, 256)               524544    
                                                                 
 dropout_2 (Dropout)         (None, 256)               0         
                                                                 
 dense_5 (Dense)             (None, 1)                 257       
                                                                 
Total params: 29,046,520
Trainable params: 528,897
Non-trainable params: 28,517,623
____________________________________

In [14]:
batch_size=32
## Training Data
train_gen = ImageDataGenerator(rescale=1/255, validation_split=0.1)
train_set = train_gen.flow_from_directory(train_path, target_size=img_size[:-1], batch_size=batch_size, shuffle=True, 
                                          seed=seed, subset='training', class_mode='binary')
# Validation data
val_set = train_gen.flow_from_directory(train_path, target_size=img_size[:-1], batch_size=batch_size, shuffle=True, 
                                        seed=seed, subset='validation', class_mode='binary')
## test data
test_gen = ImageDataGenerator(rescale=1/255)
test_set = test_gen.flow_from_directory(test_path, target_size=img_size[:-1], batch_size=batch_size, shuffle=True, 
                                        seed=seed, class_mode='binary')

Found 18001 images belonging to 2 classes.
Found 1999 images belonging to 2 classes.
Found 5000 images belonging to 2 classes.


In [20]:
# callbacks
train_frac = 0.1 ## fraction of the train set used in each optimization step
spe = (train_frac * train_set.n) // batch_size

os.makedirs('Model/', exist_ok=True)
callback_1 = ModelCheckpoint('Model/', save_freq=int(spe), save_weights_only=True)

In [16]:
gpus = tf.config.list_physical_devices('GPU')
if len(gpus):
    #tf.config.set_visible_devices([], 'CPU') # hide the CPU
    #tf.config.set_visible_devices(gpus[0], 'GPU') # unhide potentially hidden GPU
    #tf.config.get_visible_devices()
    configure_tf_gpu()
    device = tf.config.list_logical_devices('GPU')[0]
else:
    device = tf.config.list_logical_devices('CPU')[0]

In [21]:
#with tf.device('/device:GPU:0'):
with tf.device(device.name):
    model.fit(train_set, epochs=20, callbacks=[callback_1], steps_per_epoch=spe, validation_data=val_set, validation_steps=2, verbose=1)


Epoch 1/20
55/56 [============================>.] - ETA: 0s - loss: 5.0310 - accuracy: 0.5398

UnknownError: {{function_node __wrapped__SaveV2_dtypes_765_device_/job:localhost/replica:0/task:0/device:CPU:0}} Failed to WriteFile: Model\_temp/part-00000-of-00001.data-00000-of-00001.tempstate5916014653102938877 : Espacio en disco insuficiente.
; operation in progress [Op:SaveV2]